In [1]:
import os
os.chdir('..')

In [2]:
data_folder = '../data/'
server_folder = './server/'
template_folder = './templates/'

import numpy as np
import json

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
directions = ['grad', 'hess', 'bfgs', 'bfgs-inv', 'hybrid', 'hybrid-inv']
types = ['LS', 'LS-ABS', 'TR', 'TR-ABS']

unfeasible = ['TR_grad', 'TR-ABS_grad', 'LS_hybrid', 'TR_hybrid', 'TR_bfgs-inv', 'TR-ABS_bfgs-inv', 'LS_hybrid-inv', 'TR_hybrid-inv', 'TR-ABS_hybrid-inv']

combinations = []

# Add all combinations
for d in directions:
    for t in types:
        str_ = t + '_' + d
        
        if str_ not in unfeasible:
            combinations.append(str_)

In [4]:
models = ['SM_MNL', 
          'SM_Nested',
          'LPMC_MNL_DC_S', 
          'LPMC_MNL_DC_M', 
          'LPMC_MNL_DC_L',
          'LPMC_MNL_RR_S',
          'LPMC_MNL_RR_M',
          'LPMC_MNL_RR_L',
          'LPMC_MNL_Full_S',
          'LPMC_MNL_Full_M',
          'LPMC_MNL_Full_L',
          'MTMC_MNL']

# Best against Biogeme

In [7]:
results = {}

for mod in models:    
    results[mod] = {}
    
    times = {'avg': 99999, 'std': 999, 'name': None}
    epochs = {'avg': 99999, 'std': 999, 'name': None}
    

    try:
        with open(server_folder + mod + '/results/biogeme_bfgs.json', 'r') as infile:
            data = json.load(infile)

        avg_time = np.mean(data['time'])
        avg_epochs = np.mean(data['epochs'])

        times['bio_avg'] = np.mean(data['time'])
        times['bio_std'] = np.std(data['time'])
        epochs['bio_avg'] = np.mean(data['epochs'])
        epochs['bio_std'] = np.std(data['epochs'])
        
    except FileNotFoundError:
        
        times['bio_avg'] = 100
        times['bio_std'] = 9

        epochs['bio_avg'] = 100
        epochs['bio_std'] = 9
    
    for comb in combinations:
                
        try:
            with open(server_folder + mod + '/results/' + comb + '.json', 'r') as infile:
                data = json.load(infile)

            avg_time = np.mean(data['time'])
            
            if avg_time < times['avg']:
                times['avg'] = avg_time
                times['std'] = np.std(data['time'])
                times['name'] = comb
                epochs['avg'] = np.mean(data['epochs'])
                epochs['std'] = np.std(data['epochs'])
                epochs['name'] = comb
        
        except FileNotFoundError:
            pass
        
    if times['avg'] > times['bio_avg']:
        times['ratio'] = "$\\times {:.2f}$".format(times['avg']/times['bio_avg'])
    else:
        times['ratio'] = "$\div {:.2f}$".format(times['bio_avg']/times['avg'])
        
        
    if epochs['avg'] > epochs['bio_avg']:
        epochs['ratio'] = "$\\times {:.2f}$".format(epochs['avg']/epochs['bio_avg'])
    else:
        epochs['ratio'] = "$\div {:.2f}$".format(epochs['bio_avg']/epochs['avg'])        
        
    results[mod]['time'] = times
    results[mod]['epochs'] = epochs

In [30]:
dir_to_str = {
    'hess': 'Hessian',
    'grad': 'Gradient',
    'bfgs': 'BFGS',
    'bfgs-inv': 'BFGS$^{-1}$',
    'hybrid': 'Hybrid',
    'hybrid-inv': 'Hybrid$^{-1}$'
}

In [55]:
for s in ['time', 'epochs']:
    print('Make table for {}\n'.format(s))

    print('\\begin{table}[H]')
    print('  \centering')
    print('  \\begin{tabular}{l||c|c|c|c}')
    print('    Models & \multicolumn{2}{c|}{Best algorithm} & Biogeme & Ratio \\\\ \hline \hline')

    for mod in models:

        res = results[mod][s]

        model_name = mod.replace('_', '\_')

        alg_name = res['name'].split('_')[0] + ' + ' + dir_to_str[res['name'].split('_')[1]]

        prec = ':.2f'
        if 'SM' in mod or 'DC' in mod:
            prec = ':.3f'

        val_res = ''
        if res['std'] == 0:
            val_res = '${:d}$'.format(int(res['avg']))
        else:
            str_res = '${{{}}} \pm {{{}}}$'.format(prec, prec)
            val_res = str_res.format(res['avg'], res['std'])

        val_bio = ''
        if res['bio_std'] == 0:
            val_bio = '${:d}$'.format(int(res['bio_avg']))
        else:
            str_res = '${{{}}} \pm {{{}}}$'.format(prec, prec)
            val_bio = str_res.format(res['bio_avg'], res['bio_std'])

        print('    \\texttt{{{}}} & {} & '.format(model_name, alg_name) + val_res + ' & ' + val_bio + ' & ' + res['ratio'] + ' \\\\')

    print('  \end{tabular}')
    print('  \caption{CAPTION => ' + s + ' }')
    print('\end{table}')
    print('\n')

Make table for time

\begin{table}[H]
  \centering
  \begin{tabular}{l||c|c|c|c}
    Models & \multicolumn{2}{c|}{Best algorithm} & Biogeme & Ratio \\ \hline \hline
    \texttt{SM\_MNL} & TR + Hessian & $0.113 \pm 0.005$ & $0.067 \pm 0.012$ & $\times 1.70$ \\
    \texttt{SM\_Nested} & TR + Hessian & $0.636 \pm 0.012$ & $0.517 \pm 0.071$ & $\times 1.23$ \\
    \texttt{LPMC\_MNL\_DC\_S} & TR + Hessian & $0.471 \pm 0.005$ & $1.656 \pm 0.135$ & $\div 3.52$ \\
    \texttt{LPMC\_MNL\_DC\_M} & TR + Hessian & $0.883 \pm 0.012$ & $2.794 \pm 0.133$ & $\div 3.16$ \\
    \texttt{LPMC\_MNL\_DC\_L} & TR + Hessian & $1.276 \pm 0.010$ & $4.320 \pm 0.168$ & $\div 3.39$ \\
    \texttt{LPMC\_MNL\_RR\_S} & LS-ABS + Hybrid$^{-1}$ & $13.85 \pm 1.01$ & $62.11 \pm 0.50$ & $\div 4.48$ \\
    \texttt{LPMC\_MNL\_RR\_M} & LS-ABS + Hybrid$^{-1}$ & $23.73 \pm 1.23$ & $120.09 \pm 0.65$ & $\div 5.06$ \\
    \texttt{LPMC\_MNL\_RR\_L} & LS-ABS + Hybrid$^{-1}$ & $31.54 \pm 2.02$ & $167.51 \pm 0.40$ & $\div 5.31$ \\
    

# Tables in the appendix

In [156]:
results = {}

for mod in models:    
    results[mod] = {}
    
    times = {'avg': 99999, 'name': None}
    epochs = {'avg': 99999, 'name': None}
    
    for comb in combinations:
                
        with open(server_folder + mod + '/results/' + comb + '.json', 'r') as infile:
            data = json.load(infile)
        
        results[mod][comb] = {}
        
        avg_time = np.mean(data['time'])
        std_time = np.std(data['time'])
        
        results[mod][comb]['time'] = {'avg': avg_time, 'std': std_time, 'best': False}
        
        if avg_time < times['avg']:
            times['avg'] = avg_time
            times['name'] = comb

        avg_epochs = np.mean(data['epochs'])
        std_epochs = np.std(data['epochs'])
        
        results[mod][comb]['epochs'] = {'avg': avg_epochs, 'std': std_epochs}
            
        avg_ll = np.mean(data['LL'])
        std_ll = np.std(data['LL'])
        
        results[mod][comb]['LL'] = {'avg': avg_ll, 'std': std_ll}
            
    results[mod][times['name']]['time']['best'] = True

In [196]:
def format(tmp, info, prec):
    val = ''
    
    if info is 'LL':
        if tmp[info]['std'] == 0:
            val = '{:.2f}'.format(tmp[info]['avg'])
        else:
            val = '{:.2f}'.format(tmp[info]['avg']) + ' \pm \\num{' + '{:.1E}'.format(tmp[info]['std']) + '}'
        val = '($\mathcal{L} = ' + val + '$)'
    else:
        if tmp[info]['std'] < 1e-8:
            val = '{:d}'.format(int(tmp[info]['avg']))
        else:
            str_res = '{{{}}} \pm {{{}}}'.format(prec, prec)
            val = str_res.format(tmp[info]['avg'], tmp[info]['std'])
            
    if info is 'time':
        val = '\\boldmath$' + val + '$ [s]'
        
    if info is 'epochs':
        val = '$\mathit{' + val + '}$ [ep]'
        
    if tmp['time']['best']:
        val = '\\cellcolor{black!15}' + val
        #val = '\\red{' + val + '}'
        
    if tmp['epochs']['avg'] >= 1000:
        val = '\gray{' + val + '}'
        
    return val

In [197]:
for mod in models:
    
    res = results[mod]
    
    prec = ':.2f'
    if 'SM' in mod or 'DC' in mod:
        prec = ':.3f'
    
    print('\\begin{landscape}')
    print('\\begin{table}[H]')
    print('  \centering')
    print('  \\begin{tabular}{l||c|c|c|c}')
    print('    & LS & LS-ABS & TR & TR-ABS \\\\ \hline\hline')
        
    val1 = format(res['LS_grad'], 'time', prec)    
    val2 = format(res['LS-ABS_grad'], 'time', prec)
    
    print('    \\multirow{3}*{Gradient} & ' + val1 + ' & ' + val2 + ' & \multirow{3}*{\\xmark} & \multirow{3}*{\\xmark} \\\\')
    
    val1 = format(res['LS_grad'], 'epochs', prec)    
    val2 = format(res['LS-ABS_grad'], 'epochs', prec)
    
    print('    & ' + val1 + ' & ' + val2 + ' && \\\\')
    
    val1 = format(res['LS_grad'], 'LL', prec)    
    val2 = format(res['LS-ABS_grad'], 'LL', prec)
    
    print('    & ' + val1 + ' & ' + val2 + ' && \\\\ \hline')
    
    val1 = format(res['LS_hess'], 'time', prec)    
    val2 = format(res['LS-ABS_hess'], 'time', prec)
    val3 = format(res['TR_hess'], 'time', prec)
    val4 = format(res['TR-ABS_hess'], 'time', prec)

    print('    \multirow{3}*{Hessian} & ' + val1 + ' & ' + val2 + ' & ' + val3 + ' & ' + val4 + ' \\\\')
    
    val1 = format(res['LS_hess'], 'epochs', prec)    
    val2 = format(res['LS-ABS_hess'], 'epochs', prec)
    val3 = format(res['TR_hess'], 'epochs', prec)
    val4 = format(res['TR-ABS_hess'], 'epochs', prec)
    
    print('    & ' + val1 + ' & ' + val2 + ' & ' + val3 + ' & ' + val4 + ' \\\\')
    
    val1 = format(res['LS_hess'], 'LL', prec)    
    val2 = format(res['LS-ABS_hess'], 'LL', prec)
    val3 = format(res['TR_hess'], 'LL', prec)
    val4 = format(res['TR-ABS_hess'], 'LL', prec)
    
    print('    & ' + val1 + ' & ' + val2 + ' & ' + val3 + ' & ' + val4 + ' \\\\ \hline')
    
    val1 = format(res['LS_bfgs'], 'time', prec)    
    val2 = format(res['LS-ABS_bfgs'], 'time', prec)
    val3 = format(res['TR_bfgs'], 'time', prec)
    val4 = format(res['TR-ABS_bfgs'], 'time', prec)

    print('    \multirow{3}*{BFGS} & ' + val1 + ' & ' + val2 + ' & ' + val3 + ' & ' + val4 + ' \\\\')
    
    val1 = format(res['LS_bfgs'], 'epochs', prec)    
    val2 = format(res['LS-ABS_bfgs'], 'epochs', prec)
    val3 = format(res['TR_bfgs'], 'epochs', prec)
    val4 = format(res['TR-ABS_bfgs'], 'epochs', prec)

    print('    & ' + val1 + ' & ' + val2 + ' & ' + val3 + ' & ' + val4 + ' \\\\')
    
    val1 = format(res['LS_bfgs'], 'LL', prec)    
    val2 = format(res['LS-ABS_bfgs'], 'LL', prec)
    val3 = format(res['TR_bfgs'], 'LL', prec)
    val4 = format(res['TR-ABS_bfgs'], 'LL', prec)
    
    print('    & ' + val1 + ' & ' + val2 + ' & ' + val3 + ' & ' + val4 + ' \\\\ \hline')
    
    val1 = format(res['LS_bfgs-inv'], 'time', prec)    
    val2 = format(res['LS-ABS_bfgs-inv'], 'time', prec)

    print('    \multirow{3}*{BFGS$^{-1}$} & ' + val1 + ' & ' + val2 + ' & \multirow{3}*{\\xmark} & \multirow{3}*{\\xmark} \\\\')
    
    val1 = format(res['LS_bfgs-inv'], 'epochs', prec)    
    val2 = format(res['LS-ABS_bfgs-inv'], 'epochs', prec)
    
    print('    & ' + val1 + ' & ' + val2 + ' && \\\\')
    
    val1 = format(res['LS_bfgs-inv'], 'LL', prec)    
    val2 = format(res['LS-ABS_bfgs-inv'], 'LL', prec)
    
    print('    & ' + val1 + ' & ' + val2 + ' && \\\\ \hline')
    
    val2 = format(res['LS-ABS_hybrid'], 'time', prec)    
    val4 = format(res['TR-ABS_hybrid'], 'time', prec)

    print('    \multirow{3}*{Hybrid} & \multirow{3}*{\\xmark} & ' + val2 + ' & \multirow{3}*{\\xmark} & ' + val4 + ' \\\\')
    
    val2 = format(res['LS-ABS_hybrid'], 'epochs', prec)    
    val4 = format(res['TR-ABS_hybrid'], 'epochs', prec)
    
    print('    && ' + val2 + ' && ' + val4 + ' \\\\')
    
    val2 = format(res['LS-ABS_hybrid'], 'LL', prec)    
    val4 = format(res['TR-ABS_hybrid'], 'LL', prec)
    
    print('    && ' + val2 + ' && ' + val4 + ' \\\\ \hline')
    
    val2 = format(res['LS-ABS_hybrid-inv'], 'time', prec)    

    print('    \multirow{3}*{Hybrid$^{-1}$} & \multirow{3}*{\\xmark} & ' + val2 + ' & \multirow{3}*{\\xmark} & \multirow{3}*{\\xmark} \\\\')
    
    val2 = format(res['LS-ABS_hybrid-inv'], 'epochs', prec)    
    
    print('    && ' + val2 + ' && \\\\')
    
    val2 = format(res['LS-ABS_hybrid-inv'], 'LL', prec)    
    
    print('    && ' + val2 + ' && \\\\ \hline')
    
    print('  \end{tabular}')
    print('  \caption{Optimization of model \\texttt{' + mod.replace('_', '\_') +'}. Average and std dev on 20 runs. Top row is the time in seconds, middle row is the number of epochs, and bottom row, in parentheses, is the log-likelihood. Max of 1000 epochs. Text in grey means that the algo did not converge. Background in grey is the best algorithm in terms of time.}')
    print('\end{table}')
    print('\end{landscape}')
    print('\n')
          

\begin{landscape}
\begin{table}[H]
  \centering
  \begin{tabular}{l||c|c|c|c}
    & LS & LS-ABS & TR & TR-ABS \\ \hline\hline
    \multirow{3}*{Gradient} & \boldmath$0.908 \pm 0.018$ [s] & \boldmath$1.014 \pm 0.153$ [s] & \multirow{3}*{\xmark} & \multirow{3}*{\xmark} \\
    & $\mathit{39}$ [ep] & $\mathit{30.849 \pm 6.662}$ [ep] && \\
    & ($\mathcal{L} = 5331.25 \pm \num{1.8E-12}$) & ($\mathcal{L} = 5331.25 \pm \num{6.2E-08}$) && \\ \hline
    \multirow{3}*{Hessian} & \boldmath$0.240 \pm 0.007$ [s] & \boldmath$0.464 \pm 0.031$ [s] & \cellcolor{black!15}\boldmath$0.113 \pm 0.005$ [s] & \boldmath$0.300 \pm 0.019$ [s] \\
    & $\mathit{8}$ [ep] & $\mathit{11.248 \pm 0.783}$ [ep] & \cellcolor{black!15}$\mathit{5}$ [ep] & $\mathit{7.249 \pm 0.282}$ [ep] \\
    & ($\mathcal{L} = 5331.25$) & ($\mathcal{L} = 5331.25 \pm \num{2.3E-08}$) & \cellcolor{black!15}($\mathcal{L} = 5331.25 \pm \num{1.8E-12}$) & ($\mathcal{L} = 5331.25 \pm \num{4.7E-12}$) \\ \hline
    \multirow{3}*{BFGS} & \boldmath$